In [1]:
import pandas as pd
from IPython.display import JSON
import pylab
import json
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-whitegrid') # TODO: what did this do?
import numpy as np
from IPython.core import display
import psycopg2
import os
homedir = os.getenv("HOME")
with open(homedir + "/.pgpass",'r') as pgfile:
    pgdat = pgfile.read().strip().split(":")
    pgurl = pgdat[0]
    pgport = pgdat[1]
    pgdb = pgdat[2]
    pguser = pgdat[3]
    pgpass = pgdat[4]

%load_ext sql
from sqlalchemy import create_engine
# sql info: https://github.com/catherinedevlin/ipython-sql
# on error `pip install ipython-sql`
#uncomment for remote
#pgport=3333
pgport=5433
engine = create_engine('postgresql://postgres@localhost:%i/postgres' % pgport)
db = psycopg2.connect(host=pgurl, database=pgdb, user=pguser,password=pgpass,port=pgport)

In [2]:
def qry(q):
  return pd.read_sql_query(con = db, sql = q)
# pd.set_option("max_colwidth", None)
# pd.set_option('display.max_columns', None)
pd.set_option("display.latex.repr", False)
pd.set_option("display.latex.escape", True)

def iDisplay(v):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, "max_colwidth", None):
        display.display(v)


Count number of uploaded APK files

In [3]:
%%sql postgresql://postgres@localhost:$pgport/postgres
select
    (select count(*) from apks where apkname not like 'jar_%' and apkname not like 'spec_%') as apk_count,
    (select count(*) from apks) as blob_count, 
    (select count(*) from inputs) as input_count, 
    (select count(*) from jobs) as job_count, 
    (select count(*) from resultdata) as result_data_count, 
    (select count(*) from results) as result_count

1 rows affected.


apk_count,blob_count,input_count,job_count,result_data_count,result_count
55,57,1,1410,1052,1084


In [4]:
%%sql -- count locations by pattern
select count(*), cast(jobs.config::json->'tag'->'heuristicType' as varchar) as category from jobs group by category

 * postgresql://postgres@localhost:5433/postgres
5 rows affected.


count,category
55,"""SensitiveDerefFieldCausedSynch"""
357,"""Disallow.I_CIEnter_Dialogdismiss"""
163,"""Disallow.I_CIEnter_AsyncTaskexecute"""
33,"""SensitiveDerefFieldCausedFinish"""
802,"""SensitiveDerefCallinCaused"""


In [53]:
%%sql --    # (select count(*) from jobs where status = 'new') as New_Jobs, # new jobs takes a while to run, only add if needed
select 
    (select count(*) from jobs where status = 'completed') as Completed_Jobs,
    (select count(*) from jobs where status = 'new') as New_Jobs,
    (select count(*) from jobs where status = 'paused') as Paused_Jobs,
    (select count(*) from jobs) as Total_Jobs,
    (select count(*) from jobs where status = 'acquired') as Acquired_jobs,
    (select count(id) from jobs 
       where stderr like '%Exception%no active body present for method%') as Body_Exception,
    (select count(id) from jobs where stderr like '%an implementation is missing%') as unimplemented,
    (select count(id) from jobs where stderr like '%Exception%') as Exception,
    (select count(id) from jobs where stderr like '%no libz3java in java.library.path%') as path_exception,
    (select count(id) from jobs where stderr like '%Failed to parse reflective type reference%') as Refl_exception,
    (select count(id) from jobs where status like 'failed%' and status not like 'failed: Subprocess Timeout') as worker_failure,
    (select count(id) from jobs where status like 'failed: Subprocess Timeout') as subprocess_timeout,
    (select count(id) from jobs where stdout like '%empty pred location%') as possible_cg_unsound,
    (select count(id) from jobs where stderr like '%OutOfMemory%') as memory
 

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


completed_jobs,new_jobs,paused_jobs,total_jobs,acquired_jobs,body_exception,unimplemented,exception,path_exception,refl_exception,worker_failure,subprocess_timeout,possible_cg_unsound,memory
1052,0,0,1410,0,0,0,0,0,0,15,343,0,0


In [6]:
#Estimate time until completion of all jobs

totTime = %sql select now() - min(started) from jobs;
completedJobs = %sql select count(*) from jobs where status = 'completed';
timeoutJobs = %sql select count(*) from jobs where status like 'failed%';
newJobs = %sql select count(*) from jobs where status = 'new';
avgJobTime = totTime[0][0] / (completedJobs[0][0] + timeoutJobs[0][0])
print("average time per job: " + str(avgJobTime))
print("est remaining time: " + str(avgJobTime * newJobs[0][0]))

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
 * postgresql://postgres@localhost:5433/postgres
1 rows affected.
average time per job: 0:05:00.784273
est remaining time: 0:00:00


In [7]:
%%sql 
select count(*) from jobs where jobs.status like '%existing results data nonempty%'


 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


count
0


Print jobs and statuses

In [8]:
%%sql
select owner,count(*) as jobcount from jobs where status ='completed' group by owner order by jobcount;

 * postgresql://postgres@localhost:5433/postgres
33 rows affected.


owner,jobcount
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia uuid: b8998cf9-a271-4a3d-b22e-6222aeed5d81,1
Linux Loki 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loki uuid: 6c6d05ef-0794-4d03-974a-f73eb2b2e6e6,1
Linux Loaner 5.19.0-35-generic #36-Ubuntu SMP PREEMPT_DYNAMIC Fri Feb 3 18:36:56 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loaner uuid: 6951971b-52ba-49f2-95b3-06dc525a3871,2
Linux historia 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia uuid: be650c3e-03aa-4bf3-8eae-c3bc405a285a,2
Linux historia-uc-5 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia-uc-5 uuid: e7cd0d8d-1637-4742-b51c-8f49d87d202f,4
Linux Loaner 5.19.0-35-generic #36-Ubuntu SMP PREEMPT_DYNAMIC Fri Feb 3 18:36:56 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: Loaner uuid: b12b6f81-4e78-4812-ae54-0edc5af5e5c9,5
Linux historia3 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia3 uuid: 57205db1-53fd-4f30-a496-db980d6aa9fe,6
Linux historia-uc-5 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia-uc-5 uuid: f1e253b8-306a-4868-aa1a-f4f4ebbb32e3,8
Linux historia3 5.15.0-67-generic #74-Ubuntu SMP Wed Feb 22 14:14:39 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia3 uuid: ef3e6906-efa6-4195-9c13-1808a1e81372,10
Linux historia-uc-5 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux hostname: historia-uc-5 uuid: 01ce0ac1-913d-4125-b1fa-cf2a4faadd9d,13


In [9]:
%%sql
select 
  (select avg(ended - started) from jobs where owner like '%Loki%' and status = 'completed') as Loki,
    (select avg(ended - started) from jobs where owner like '%Loan%' and status = 'completed') as Loan,
    (select avg(ended - started) from jobs where owner like '% historia %' or owner like '%historia3%' and status = 'completed') as zen3,
    (select avg(ended - started) from jobs where owner like '%historia-uc%' and status = 'completed') as cascadelake_r

 * postgresql://postgres@localhost:5433/postgres
1 rows affected.


loki,loan,zen3,cascadelake_r
0:11:06.082153,0:13:12.974776,0:15:39.948423,0:19:35.193863


In [10]:
# find exceptions
#%%sql 
#select replace(cast(stderr as varchar),'$','\$') from jobs where stderr like '%Exception%' limit 1;
outs = %sql select stderr,config from jobs where stderr like '%Exception%' and stderr not like '%OutOfMemory%' order by random() limit 1;
for out in outs:
    print(out[1])
    print(out[0])


 * postgresql://postgres@localhost:5433/postgres
0 rows affected.


In [11]:
%%sql -- look at results by tag
select cast(results.result::json->'summary' as varchar) as result, 
    cast(jobs.config::json->'tag'->'heuristicType' as varchar) as category, 
    count(*)
    from results inner join jobs on results.jobid=jobs.id where cast(jobs.config::json->'tag'->'heuristicType' as varchar)  like '%execute%'
    group by cast(results.result::json->'summary' as varchar),
    cast(jobs.config::json->'tag'->'heuristicType' as varchar)

 * postgresql://postgres@localhost:5433/postgres
2 rows affected.


result,category,count
"""\""Ijava.lang.IllegalArgumentException: cannot union primitive type set with: PrimTypeSet(byte)\""""","""Disallow.I_CIEnter_AsyncTaskexecute""",3
"""\""Unreachable\""""","""Disallow.I_CIEnter_AsyncTaskexecute""",168


In [45]:
%%sql -- results of SensitiveDerefFieldCausedSynch count
select count(*),cast(results.result::json->'summary' as varchar) from jobs inner join results on jobs.id=results.jobid 
where cast(jobs.config::json->'tag'->'heuristicType' as varchar) like '%SensitiveDerefFieldCausedSynch%' 
group by cast(results.result::json->'summary' as varchar)


 * postgresql://postgres@localhost:5433/postgres
3 rows affected.


count,varchar
5,"""\""Ijava.lang.IllegalArgumentException: cannot union primitive type set with: PrimTypeSet(byte)\"""""
5,"""\""Timeout\"""""
44,"""\""Unreachable\"""""


In [49]:
cfg = %sql select jobs.config from jobs inner join results on jobs.id=results.jobid where cast(jobs.config::json->'tag'->'heuristicType' as varchar) like '%SensitiveDerefFieldCausedSynch%'  and cast(results.result::json->'summary' as varchar) like '%Unreach%'

 * postgresql://postgres@localhost:5433/postgres
44 rows affected.


In [51]:
cfg[2][0]

'{"apkPath":"${baseDir}/fdroid/org.sipdroid.sipua/6.3beta/apk/org.sipdroid.sipua_134.apk","outFolder":["${baseDirOut}/org.sipdroid.sipua"],"specSet":"{\\"specs\\":[],\\"disallow\\":[{\\"univQuant\\":[{\\"$type\\":\\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\\",\\"n\\":\\"a\\"},{\\"$type\\":\\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\\",\\"n\\":\\"f\\"}],\\"existQuant\\":[],\\"pred\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.Or\\",\\"l1\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.NS\\",\\"i1\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\\",\\"mt\\":{\\"$type\\":\\"edu.colorado.plv.bounder.ir.CBExit\\"},\\"signatures\\":{\\"$type\\":\\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\\",\\"baseSubtypeOf\\":[\\"android.app.Fragment\\",\\"androidx.fragment.app.Fragment\\",\\"android.support.v4.app.Fragment\\"],\\"signatureMatcher\\":\\"void onDestroy\\\\\\\\(\\\\\\\\)\\",\\"

In [50]:
#"BaseLogcatActivity","WikivoyageExploreActivity","GpxSelectionHelper","WikiArticleHelper", "DataStorageHelper"
JSON([json.loads(c[0]) for c in cfg])

<IPython.core.display.JSON object>

In [37]:
print([c[0] for c in cfg if "executeWithParams" in c[0]][0])

{"apkPath":"${baseDir}/fdroid/net.osmand.plus/4.3.8/apk/net.osmand.plus_4308.apk","outFolder":["${baseDirOut}/net.osmand.plus"],"specSet":"{\"specs\":[],\"disallow\":[{\"univQuant\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"t\"}],\"existQuant\":[],\"pred\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIExit\"},\"signatures\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.SubClassMatcher\",\"baseSubtypeOf\":[\"android.os.AsyncTask\"],\"signatureMatcher\":\".*AsyncTask execute\\\\(.*\\\\)\",\"ident\":\"AsyncTask_execute\"},\"lsVars\":[{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.TopVal\"},{\"$type\":\"edu.colorado.plv.bounder.symbolicexecutor.state.NamedPureVar\",\"n\":\"t\"}]},\"target\":{\"$type\":\"edu.colorado.plv.bounder.lifestate.LifeState.OAbsMsg\",\"mt\":{\"$type\":\"edu.colorado.plv.bounder.ir.CIEnter\"},\"signatures\":{\"$type\":\"edu.colo

In [ ]:
%%sql
select cast(results.result::json->'summary' as varchar) as summary, cast(jobs.config::json->'apkPath' as varchar) as benchmark,count(*) from results inner join jobs on jobs.id=results.jobid group by summary,benchmark order by benchmark,summary desc

Get failing apks

In [ ]:
%%sql
select cast(config::json->'apkPath' as text) from jobs 
        where stderr like '%Exception%' group by cast(config::json->'apkPath' as text) ;

In [ ]:
%%sql
select cast(config::json->'apkPath' as text) as app,count(*) as count from jobs group by app order by count desc;

In [ ]:
%%sql 
select cast(result::json->'summary' as varchar) as summary, cast(result::json->'maxPathCh' as varchar) as ch,count(*),avg(querytime),max(querytime) from results group by summary,ch order by summary,ch;

In [ ]:
%%sql 
select count(*), cast(results.result::json->'summary' as varchar),cast(jobs.config::json->'tag' as varchar) as tag 
from jobs inner join results on jobs.id=results.jobid group by cast(jobs.config::json->'tag' as varchar),cast(results.result::json->'summary' as varchar)
order by  cast(jobs.config::json->'tag' as varchar)

In [ ]:
%%sql 
select regexp_substr(cast(jobs.config::json->'apkPath' as varchar), '[a-zA-Z]*\.[a-zA-Z]*') as benchmark, 
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute') as isDismiss, 
    cast(results.result::json->'summary' as varchar) as summary, count(*)
    from results inner join jobs on results.jobid=jobs.id 
    group by cast(jobs.config::json->'apkPath' as varchar),cast(results.result::json->'summary' as varchar),
    regexp_substr(cast(results.qry::json->'s' as varchar), 'Dialog_dismiss|execute')


In [ ]:
%%sql
select cast(result::json->'summary' as varchar) as summary, count(*) from results, group by cast(result::json->'summary' as varchar)

In [ ]:
%%sql
select id from results where cast(results.result::json->'summary' as varchar) like '%Witnessed%'

In [ ]:
cfg = %sql select jobs.config from results inner join jobs on results.jobid=jobs.id where cast(results.result::json->'summary' as varchar) like '%Witnessed%' limit 1;
cfg[0][0]

In [ ]:
%%sql
select id,cast(result::json->'summary' as varchar) as summary,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results order by random() limit 5;

Check result consistency
========================

In [ ]:
%%sql -- get workers with acquired jobs -- check for abandoned jobs
select owner,count(*) from jobs where status='acquired' group by owner;

In [ ]:
%%sql -- Find completed jobs where no results exist and no error occurred
select jobs.id from jobs where jobs.id not in (select results.jobid from results) and jobs.status = 'completed' and jobs.stderr not like '%Exception%';

In [ ]:
%%sql -- Find new/paused jobs with results
select jobs.id from jobs where jobs.id in (select results.jobid from results) and (jobs.status = 'paused' or jobs.status = 'new');

In [ ]:
%%sql -- check that each location/qry pair only has one result (should detect duplicat runs)
select count(*),qry,loc as num from results group by qry,loc order by count(*) desc limit 1;

Triage witness results
----------------------

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Witnessed%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWit.csv')

In [ ]:
witnessesq = %sql \
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results \
                where cast(result::json->'summary' as varchar) like '%Itimeout%' and cast(result::json->'maxPathCh' as varchar) like '%MultiCallback%' order by id asc;

witnessesdf = witnessesq.DataFrame()
witnessesdf.to_csv('triageWitMultiTimeout.csv')

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'line'->'str' as varchar),'$','\$') from results where id = 40;

Triage proven results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz,loc::json->'method'->>2 from results 
                where cast(result::json->'summary' as varchar) like '%Proven%' order by random() limit 10;

Triage unreachable results
--------------------------

In [ ]:
%%sql 
select id,jobid,resultdata,loc::json->'method'->>1 as clazz 
            from results where cast(result::json->'summary' as varchar) like '%Unreachable%' 
            order by random() limit 15;

In [ ]:
%%sql
select id,jobid,resultdata,loc::json->'method'->>1 as clazz, 
        replace(cast(loc::json->'method'->>0 as varchar),'$','\$') as mname, 
        replace(cast(loc::json->'line' as varchar),'$','\$') as location 
        from results where id = 15037;

In [ ]:
%%sql
select id,result,replace(cast(loc::json->'line' as varchar),'$','\$') as location 
    from results where loc like '%244 $r3%';

In [ ]:
%%sql
select replace(cast(loc as varchar),'$','\$') from results where id = 1494;

Runtime Stats
=============

In [ ]:
queryTime = %sql select querytime/60/60 from results;
tplt = queryTime.DataFrame().plot.hist(bins=16)
tplt.set_yscale('log')
# tplt.set_xscale('log')
tplt.set_xlabel('time (hrs)')
# split this into proven and witnessed

In [ ]:
%%sql
select 

In [ ]:
cfg = %sql select jobs.config from results join jobs on jobs.id = results.jobid where querytime > 6*60*60 limit 1;
if len(cfg) > 0:
    print(cfg[0][0])

In [ ]:
%%sql -- which apps cause runtimes over 2 hours?
select jobs.id, results.querytime, jobs.config::json->'apkPath' from results inner join jobs on jobs.id=results.jobid where results.querytime > 7200 order by results.querytime desc limit 200;

In [ ]:
# Job runtimes
runtimes = %sql select ended - started as time from jobs where ended is not NULL;
runtimesDF = runtimes.DataFrame()
runtimesDF['time'] = runtimesDF['time'].apply(lambda x: x.total_seconds() / (60))
tplt = runtimesDF.plot.hist(bins=32)
tplt.set_yscale('log')
tplt.set_xlabel('time (min)')

In [ ]:
# Plot job time trend
rtvt = %sql select started, ended - started as time from jobs where ended is not NULL;
rtvtDF = rtvt.DataFrame()
rtvtDF.sort_values('started', ascending=True)
fig, ax = plt.subplots()
ax.plot_date(rtvtDF['started'], rtvtDF['time'], marker='*', linestyle='')

fig.autofmt_xdate()
plt.show()


Total job time

In [ ]:
%%sql
select sum(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(ended-started) from jobs where ended is not NULL;

In [ ]:
%%sql
select avg(querytime) from results;

In [ ]:
%%sql
select now() - min(started) from jobs;

Size of postgres data
=====================

In [ ]:
%%sql
WITH RECURSIVE pg_inherit(inhrelid, inhparent) AS
    (select inhrelid, inhparent
    FROM pg_inherits
    UNION
    SELECT child.inhrelid, parent.inhparent
    FROM pg_inherit child, pg_inherits parent
    WHERE child.inhparent = parent.inhrelid),
pg_inherit_short AS (SELECT * FROM pg_inherit WHERE inhparent NOT IN (SELECT inhrelid FROM pg_inherit))
SELECT table_schema
    , TABLE_NAME
    , row_estimate
    , pg_size_pretty(total_bytes) AS total
    , pg_size_pretty(index_bytes) AS INDEX
    , pg_size_pretty(toast_bytes) AS toast
    , pg_size_pretty(table_bytes) AS TABLE
  FROM (
    SELECT *, total_bytes-index_bytes-COALESCE(toast_bytes,0) AS table_bytes
    FROM (
         SELECT c.oid
              , nspname AS table_schema
              , relname AS TABLE_NAME
              , SUM(c.reltuples) OVER (partition BY parent) AS row_estimate
              , SUM(pg_total_relation_size(c.oid)) OVER (partition BY parent) AS total_bytes
              , SUM(pg_indexes_size(c.oid)) OVER (partition BY parent) AS index_bytes
              , SUM(pg_total_relation_size(reltoastrelid)) OVER (partition BY parent) AS toast_bytes
              , parent
          FROM (
                SELECT pg_class.oid
                    , reltuples
                    , relname
                    , relnamespace
                    , pg_class.reltoastrelid
                    , COALESCE(inhparent, pg_class.oid) parent
                FROM pg_class
                    LEFT JOIN pg_inherit_short ON inhrelid = oid
                WHERE relkind IN ('r', 'p')
             ) c
             LEFT JOIN pg_namespace n ON n.oid = c.relnamespace
  ) a
  WHERE oid = parent
) a
ORDER BY total_bytes DESC limit 5;

Inspection of exceptions
========================

In [ ]:
%%sql
select id,config::json->'apkPath',replace(stderr,'$','\$') from jobs where stderr like '%Exception%';

In [ ]:
#count of total results
%sql select count(*) as Total_locations_computed from results